In [1]:
using Plots
using Interact, WebIO, Plots
using Symbolics, SymPy
using FiniteDifferences, ForwardDiff

WebIO._IJuliaInit()

# Homework

This homework asks you to compute a function's derivatives using symbolic and automatic differentiation methods. Our objective function is the follows.

\begin{align}
 f(x) = \log(x)\times \exp\left[-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2 \right].
\end{align}

Let's assume $\mu=1.5$ and $\sigma=2$. 



## Automatic Differentiations

In [2]:
μ = 1.5
σ = 2
f(x) = log(x) * exp(-0.5* ((x- μ)/ σ)^ 2)
true_f_prime(x) = 1/x *exp(-0.5* ((x- μ)/ σ)^ 2)+log(x) * exp(-0.5* ((x- μ)/ σ)^ 2) * - ((x- μ)/ σ) * 1/σ

true_f_prime (generic function with 1 method)

### Use the Taylor expansion to derive the result of logarithms on dual numbers.

Let $f(x) = log(x)$ and so $f'(x) = \frac{1}{x}$. For a dual number, it means $f(a+b\epsilon) = log(a+b\epsilon)$. Applying the Taylor expansion, we have

\begin{align}
 f(a +b\epsilon) & = f(a) + f'(a)b\epsilon = log(a) + \frac{1}{a} b\epsilon; \\
\Longrightarrow \quad log(a+b\epsilon) & = log(a) + \frac{1}{a} b\epsilon .
\end{align}

### Add methods of the division ("/") and logarithm ("log()") for dual numbers in Julia.

In [3]:
import Base: +, *, -, ^, exp, log, /

struct DualNumber{T1, T2} <: Real   
    re::T1                # differnt types so (Float64, Int64), etc., is possible
    du::T2
end

+(x::DualNumber, y::DualNumber) = DualNumber(x.re + y.re, x.du + y.du)  # dual addition
+(x::DualNumber, a::Number) = DualNumber(x.re + a, x.du)  
+(a::Number, x::DualNumber) = DualNumber(x.re + a, x.du) 

-(x::DualNumber, y::DualNumber) = DualNumber(x.re - y.re, x.du - y.du)  # dual subtraction
-(x::DualNumber, a::Number) = DualNumber(x.re - a, x.du)  
-(a::Number, x::DualNumber) = DualNumber(a-x.re , -x.du)  

*(x::DualNumber, y::DualNumber) = DualNumber(x.re*y.re, x.re*y.du + y.re*x.du)
*(x::DualNumber, a::Number) = DualNumber(a*x.re, a*x.du)
*(a::Number, x::DualNumber) = DualNumber(a*x.re, a*x.du)

function /(x::DualNumber, y::DualNumber) 
    if y.re == 0 return println("denominator's re part can't be 0")
    else
    return DualNumber(x.re/y.re,  (y.re*x.du - x.re*y.du)/(y.re*y.re) )
    end
end
function /(x::DualNumber, a::Number) 
    if a == 0 return println("a can't be 0")
    else
    return DualNumber((1/a)*x.re, (1/a)*x.du)
    end
end
function /(a::Number, x::DualNumber) 
    if x.re == 0 return println("denominator's re part can't be 0")
    else
    return DualNumber(a*(1/x.re), (- a*x.du)/(x.re*x.re))
    end
end
    
^(x::DualNumber, n::Union{Float64, Int64}) = DualNumber(x.re^n, n*x.re^(n-1)*x.du)  # cannot use n<:Real, since n is variable

log(x::DualNumber) = DualNumber(log(x.re),(1/x.re)*x.du)
exp(x::DualNumber) = DualNumber(exp(x.re), exp(x.re)*x.du)


exp (generic function with 35 methods)

In [9]:
x = DualNumber(1.2, 1)
@vars x 
fnt =diff(f(x),x)

                                                                              
                                                    2           -0.28125*(0.66
                  -0.28125*(0.666666666666667*x - 1)           e              
(0.375 - 0.25*x)*e                                   *log(x) + ---------------
                                                                              

                    2
6666666666667*x - 1) 
                     
---------------------
  x                  

In [10]:
fnt(1.2)

0.837532015534738

In [5]:
a=DualNumber(1.2, 1)
b=DualNumber(0, 1)
a/b

denominator's re part can't be 0


### Use the dual numbers you've defined to compute the derivative of $f(x)$ at $x=1.2$.

In [6]:
x = DualNumber(1.2, 1)
@show f(x).du

(f(x)).du = 0.8375320155347377


0.8375320155347377

In [7]:
true_f_prime(1.2)

0.8375320155347377

### Use Julia's package `ForwardDiff`, which implements the forward-mode auto differentiation, to compute the derivative of $f(x)$ at $x=1.2$.

In [8]:
@show ForwardDiff.derivative(f, 1.2)

ForwardDiff.derivative(f, 1.2) = 0.8375320155347377


0.8375320155347377

## Symbolic Differentiation

### Use Julia's package `SymPy` to compute the analytic derivative of $f(x)$ and evaluate the result at $x=1.2$.
- Hint: You may need to use `diff` and `subs` functions in the package.
- Remark: I intended to use `Symbolics` but find it less straightforward. You might give it a try.

In [9]:
@variables x
f(x)

log(x)*exp(-0.125((x - 1.5)^2))

In [34]:
fnt(1.2)

0.837532015534738

In [8]:
fprime(x) = Symbolics.derivative(f(x),x) 
fprime(x)|> display

exp(-0.125((x - 1.5)^2)) / x - 0.25(x - 1.5)*log(x)*exp(-0.125((x - 1.5)^2))

In [9]:
fp = Symbolics.simplify(fprime(x))
V = substitute.(fp, x => 1.2)

0.8375320155347377